In [1]:
#!/home/admin/anaconda3/bin/python3
from datetime import datetime
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import sklearn.utils
import pandas as pd
import argparse,sys
import mmutils

parser = argparse.ArgumentParser(description='Run Tpot model and generate test/train and Tpot file with comments',formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument( "--tmodel", type=str, default="light",
        help="Type of Tpot model - type None to get a list (default: %(default)s)")
#argv=parser.parse_args()

# prepend comments to csv - should be last fn called before closing
def prependComments(filename,comments):
#    print(f"prepend {filename}: {comments}")
    with open(filename,'r') as contents:
        save = contents.read()
    with open(filename,'w') as contents:
        contents.write(comments)
    with open(filename,'a') as contents:
        contents.write(save)
    return comments

# Options
max_time_mins=1
random_state=42
ifile="data/TK.csv"
generations=100
test_size=0.1
config_dict=None

tmodels=mmutils.tmodels()
tmodels["GB"]

mmutils.tmodels()["GB"]
print(tmodels)

tmodel="light"
#tmodel=argv.tmodel
try:
    print(f"Tpot search tmodels {tmodels[tmodel]}")
except:
    print(f"Unknown tmodel try one of the following")
    for i in tmodels:
        print(f"{i}={tmodels[i]}")
    sys.exit(0)

df=pd.read_csv(ifile,comment='#')
ofile=ifile.replace(".csv","_"+tmodel+".csv")
train, test = train_test_split(df, test_size=test_size,random_state=random_state)
train_file=ofile.replace(".csv","_train.csv")
train.to_csv(train_file,index = False)
print(f"Created File {train_file}")
test_file=ofile.replace(".csv","_test.csv")
print(f"Created File {test_file}")
test.to_csv(test_file,index = False)

X_train=train.drop(['kepid','koi_disposition','kepoi_name'], axis=1).to_numpy()
y_train, uniques =pd.factorize(train['koi_disposition'],sort=False)
X_test=test.drop(['kepid','koi_disposition','kepoi_name'], axis=1).to_numpy()
y_test, uniques =pd.factorize(test['koi_disposition'],sort=False)

tmodels =  {
        "light": "TPOT light",
        "DT":"sklearn.tree.DecisionTreeClassifier", # Inherently multiclass",
        "GNB":"sklearn.naive_bayes.GaussianNB", #Inherently multiclass",
        "RF":"sklearn.ensemble.RandomForestClassifier", #Inherently multiclass",  
        "LR":"sklearn.linear_model.LogisticRegression", # (setting multi_class='multinomial')",
        "KNN":"sklearn.neighbors.KNeighborsClassifier", #Inherently multiclass",
        "BNB":"sklearn.naive_bayes.BernoulliNB", #Inherently multiclass",
        "GB":"sklearn.ensemble.GradientBoostingClassifier", #Multiclass as One-Vs-The-Rest - was chosen",
        "NN":"sklearn.neural_network.MLPClassifier" #Inherently multiclass"
        } 

tmodel="RF"
#X_train,y_train,X_test,uniques 
if tmodel=="None":
    config_dict="light"
elif tmodel=="light":
    config_dict=tmodels[tmodel]
else:
    key=tmodels[tmodel]
    value="{{ }}"
    config_dict={key:{}}

#config_dict = { 'sklearn.ensemble.RandomForestClassifier': { } } - works
Tpot = TPOTClassifier(verbosity=2,generations=generations,config_dict=config_dict,max_time_mins=max_time_mins,random_state=random_state)
Tpot.fit(X_train, y_train)

Tpot_score=Tpot.score(X_test, y_test)
print(Tpot_score) # The correct score is in the tpot export file
Tpot_file=ofile.replace(".csv","_Tpot.py")
Tpot.export(Tpot_file)

date=datetime.now().strftime("%d/%m/%Y_%H:%M:%S")

mycomments=f"# ifile={ifile},ofile={ofile}, generations={generations}, config_dict={config_dict}\n"
mycomments=mycomments+f"# tmodel={tmodel}: {tmodels[tmodel]},Tpot_file={Tpot_file}, Tpot_score={Tpot_score}, date={date}\n" 
prependComments(Tpot_file,mycomments)
prependComments(train_file,mycomments)
prependComments(test_file,mycomments)


Unknown tmodel try one of the following


NameError: name 'tmodels' is not defined